## **RADI605: Modern Machine Learning**

### Assignment: Random Forests
**Romen Samuel Rodis Wabina** <br>
Student, PhD Data Science in Healthcare and Clinical Informatics <br>
Clinical Epidemiology and Biostatistics, Faculty of Medicine (Ramathibodi Hospital) <br>
Mahidol University

Note: In case of Python Markdown errors, you may access the assignment through this GitHub [Link](https://github.com/rrwabina/RADI605/blob/main/05%20Adaptive%20Boosting/scripts/assignment.ipynb)

From the given data, create a random forest with $T=3$, OOB rate = $20\%$, and initialize any additional parameters that may be needed.

| Sample M  | Gene #1       | Gene #2         | Gene #3         | Class           |
|:---------:|---------------|-----------------|-----------------|-----------------|
| 1         | -3            | 1               | 6               | 1               |
| 2         | 3             | -1              | 6               | 1               |
| 3         | 6             | 1               | 12              | 1               |
| 4         | 1             | 0               | 2               | -1              |
| 5         | 0             | 1               | 0               | -1              |

Out of the total number of samples in the data, 20% of the samples will be left out for Out-of-Bag (OOB) error calculation. So, for the given data with 5 samples, we will have 1 sample as OOB. Then, the original data will be bootstrapped to create $T = 3$ bootstrapped datasets. Suppose our OOB sample is sample 4:

| Sample M  | Gene #1       | Gene #2         | Gene #3         | Class           |
|:---------:|---------------|-----------------|-----------------|-----------------|
| 4         | 1             | 0               | 2               | -1              |

Bootstrapping: To create $T = 3$ bootstrapped datasets, we will randomly sample with replacement from the original data T times. Each bootstrapped dataset will have the same number of samples as the original data. Suppose bootstrapped dataset $D_1$ is the following:

| Sample M  | Gene #1       | Gene #2         | Gene #3         | Class           |
|:---------:|---------------|-----------------|-----------------|-----------------|
| 1         | -3            | 1               | 6               | 1               |
| 2         | 3             | -1              | 6               | 1               |
| 4         | 1             | 0               | 2               | -1              |
| 5         | 0             | 1               | 0               | -1              |
| 4         | 1             | 0               | 2               | -1              |

To calculate the Gini index and Gini gain for bootstrapped $D_1$, we will first calculate the Gini index for each feature (Gene #1, Gene #2, Gene #3) and use the Gini gain to determine the best feature to split the data at the root node.

### Root Node: The Gini index measures the impurity of a set of samples. For a binary classification problem, the Gini index can be calculated as follows:

\begin{equation*}
\begin{aligned}
    \text{Gini}(D_1)    &= 1 - \sum_{j = 1}^{L}p^2_j \\
                        &= 1 - (0.4)^2 - (0.4)^2 \\
                        &= 1 - 0.32 \\
    \text{Gini}(D_1)    &= 0.48
\end{aligned}
\end{equation*}

### Splits on Gene #1
Next, we'll calculate the gini gain for Gene #1. The split on Gene #1 is -3.
- $\text{Class 1}  \rightarrow \frac{0}{1} = 0$ and $\text{Class -1} \rightarrow \frac{1}{1} = 1 $

Therefore, 
\begin{equation*}
\begin{aligned}
    \text{Gini index split}(D_1)    &= 1 - \sum_{j = 1}^{L}p^2_j \\
                              &= 1 - (0)^2 - (1)^2 \\
                              &= 1 - 1 \\
    \text{Gini index split}(D_1)    &= 0
\end{aligned}
\end{equation*}

\begin{equation*}
\begin{aligned}
    \text{Gini Gain split}(D_1)    &= \text{Gini}(D_1) - \sum_{i = 1}^{n} \frac{|D_{1i}|}{|D_1|} \text{Gini}(D_1i)\\
                                   &= 0.48 - (1/5) \cdot 0 - (4/5) \cdot 0.48 \\
    \text{Gini Gain split}(D_1)    &= 0.24
\end{aligned}
\end{equation*}

We'll repeat the calculation for the gini gain for Gene #1 where the split on Gene #1 is 3.
- $\text{Class 1}  \rightarrow \frac{1}{2} = 0.5$ and $\text{Class -1} \rightarrow \frac{0}{2} = 0 $

Therefore, 
\begin{equation*}
\begin{aligned}
    \text{Gini index split}(D_1)    &= 1 - \sum_{j = 1}^{L}p^2_j \\
                                    &= 1 - (0.5)^2 - (0)^2 \\
    \text{Gini index split}(D_1)    &= 0.5
\end{aligned}
\end{equation*}

\begin{equation*}
\begin{aligned}
    \text{Gini Gain split}(D_1)    &= \text{Gini}(D_1) - \sum_{i = 1}^{n} \frac{|D_{1i}|}{|D_1|} \text{Gini}(D_1i)\\
                                   &= 0.48 - (2/5) \cdot 0 - (3/5) \cdot 0.48 \\
    \text{Gini Gain split}(D_1)    &= 0.072
\end{aligned}
\end{equation*}

Finally, calculate the gini gain for Gene #1 where the split on Gene #1 is 1.
- $\text{Class 1}  \rightarrow \frac{1}{2} = 0.5$ and $\text{Class -1} \rightarrow \frac{1}{2} = 0.5 $

Therefore, 
\begin{equation*}
\begin{aligned}
    \text{Gini index split}(D_1)    &= 1 - \sum_{j = 1}^{L}p^2_j \\
                                    &= 1 - (0.5)^2 - (0.5)^2 \\
    \text{Gini index split}(D_1)    &= 0.5
\end{aligned}
\end{equation*}

\begin{equation*}
\begin{aligned}
    \text{Gini Gain split}(D_1)    &= \text{Gini}(D_1) - \sum_{i = 1}^{n} \frac{|D_{1i}|}{|D_1|} \text{Gini}(D_1i)\\
                                   &= 0.48 - (2/5) \cdot 0.5 - (3/5) \cdot 0.48 \\
    \text{Gini Gain split}(D_1)    &= 0.072
\end{aligned}
\end{equation*}

The split for value 0 in Gene #1 was not included because, in the calculation of gini gain, it resulted in a negative gain value. The CART algorithm used in Random Forest only chooses the split that results in the highest positive gain value. As a result, a split for value 0 in Gene #1 was not included. Since Gene #1 has the highest gini gain when split on value = $-3$, we'll use that split for the root of the tree.
The tree would look like:

           -3
           /  \
         -1   1
        /  \  / \
      1    -1    1
For Gene #2:

Split at -1:
- gini index for Class = 1: 1 - (3/4)^2 - (1/4)^2 = 0.375
- gini index for Class = -1: 1 - (2/4)^2 - (2/4)^2 = 0.5
- gini gain: 0.375 - (4/8) * 0.375 - (4/8) * 0.5 = 0.0625

Split at 1:
- gini index for Class = 1: 1 - (2/3)^2 - (1/3)^2 = 0.44444444
- gini index for Class = -1: 1 - (2/5)^2 - (3/5)^2 = 0.48
- gini gain: 0.44444444 - (3/8) * 0.44444444 - (5/8) * 0.48 = 0.01111111

For Gene #3:

Split at 2:
- gini index for Class = 1: 1 - (2/2)^2 - (0/2)^2 = 0
- gini index for Class = -1: 1 - (2/2)^2 - (0/2)^2 = 0
- gini gain: 0 - (2/8) * 0 - (6/8) * 0 = 0

Split at 6:
- gini index for Class = 1: 1 - (2/2)^2 - (0/2)^2 = 0
- gini index for Class = -1: 1 - (2/6)^2 - (4/6)^2 = 0.48
- gini gain: 0 - (2/8) * 0 - (6/8) * 0.48 = -0.12

The highest gini gain was obtained from Gene #2 with a split at -1, so the first split of the tree will be based on Gene #2 and split at -1.

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

from numpy import mean
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from mlxtend.plotting import plot_learning_curves
from statsmodels.stats.outliers_influence import variance_inflation_factor

import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
def convert_binary(df, columns = ['Risk1Yr']):
    for column in columns:
        df[column] = df[column].apply(lambda x: 0 if x == 'F' else 1)
    return df

def load_thoracic(path = '../data/ThoraricSurgery.csv'):
    data = pd.read_csv(path)
    data = data[data.columns[2:]]
    data = data.drop(['PRE6', 'PRE14'], axis = 1)
    label_columns = data.columns[2:12]
    data = convert_binary(data, columns = ['Risk1Yr'])
    data = convert_binary(data, columns = label_columns)
    include_columns = data.columns[0:-1]
    X, y = data[include_columns], data['Risk1Yr']
    X, y = X.to_numpy(), y.to_numpy()
    y[y == 0] = -1
    return X, y, data

X, y, data = load_thoracic()

When features are highly correlated, they contain redundant information, and the Random Forest may use the same feature in many of the trees, leading to overfitting. Additionally, highly correlated features can cause instability in the feature importance scores and make it difficult to interpret the results.

It's always a good idea to check for highly correlated features before training a Random Forest and remove or combine them as needed. This can be done using techniques such as principal component analysis (PCA) or feature selection. By reducing the number of highly correlated features, you can simplify the model and make it more interpretable, and you may also see an improvement in performance.


In [3]:
def convert_binary(df, columns = ['Risk1Yr']):
    for column in columns:
        df[column] = df[column].apply(lambda x: -1 if x == 'F' else 1)
    return df

def load_thoracic(path = '../data/ThoraricSurgery.csv'):
    data = pd.read_csv(path)
    data = data[data.columns[2:]]
    data = data.drop(['PRE6', 'PRE14'], axis = 1)
    label_columns = data.columns[2:12]
    data = convert_binary(data, columns = ['Risk1Yr'])
    data = convert_binary(data, columns = label_columns)
    include_columns = data.columns[0:-1]
    X, y = data[include_columns], data['Risk1Yr']
    X, y = X.to_numpy(), y.to_numpy()
    y[y == 0] = -1
    return X, y, data

def split_data(X, y, pca_included = False, smote_included = False):
    print('Model Assumptions:')
    if pca_included is True:
        print('\t The dataset used PCA for dimensionality reduction.')
        pca = PCA(n_components = 8)
        X = pca.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    if smote_included is True:
        print('\t The dataset used SMOTE to rectify class imbalance.')
        smote = SMOTE(sampling_strategy = 'minority', k_neighbors = 5, random_state = 42)
        X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    else:
        print('\t The dataset did NOT use SMOTE.')
        X_resampled, y_resampled = X_train, y_train 
    scaler  = StandardScaler()
    X_resampled = scaler.fit_transform(X_resampled)
    X_test  = scaler.transform(X_test)
    return X_resampled, y_resampled, X_test, y_test

def init_parameters():
    param_grid = { 
                'n_estimators': [10, 2000], 
                'criterion': ['gini', 'entropy'],
                'max_depth': np.arange(1, 20),
                'min_samples_split': np.arange(1, 5)
              }
    return param_grid

def validation(rsearch, X_test, y_test):
    predictions = rsearch.predict(X_test)
    print('Confusion Matrix: ')
    print(confusion_matrix(y_test, predictions))
    print(classification_report(y_test, predictions))
    return predictions

def train_randomforest(X_train, y_train, X_test, y_test, use_randomsearch = False, print_params = True):
    param_grid = init_parameters()
    if use_randomsearch is True:
        print('\t This model has been cross-validated through Random Search.')
        rsearch = RandomizedSearchCV(estimator = RandomForestClassifier(), 
                                     param_distributions = param_grid, 
                                     cv = 10, n_iter = 10)
        rsearch.fit(X_train, y_train)
        if print_params is True:
            print(rsearch.best_params_)
        predictions = validation(rsearch, X_test, y_test)
    else:
        print('\t This model did NOT cross-validate through Random Search.')
        rsearch = RandomForestClassifier(criterion = 'gini', n_estimators = 100, 
                                         max_depth = 9, min_samples_split = 4)
        rsearch.fit(X_train, y_train)
        predictions = validation(rsearch, X_test, y_test)
    return predictions 

In [4]:
X, y, data = load_thoracic()
X_train, y_train, X_test, y_test = split_data(X, y, pca_included = False, smote_included = False)
predictions = train_randomforest(X_train, y_train, X_test, y_test, use_randomsearch = False)

Model Assumptions:
	 The dataset did NOT use SMOTE.
	 This model did NOT cross-validate through Random Search.
Confusion Matrix: 
[[75  0]
 [19  0]]
              precision    recall  f1-score   support

          -1       0.80      1.00      0.89        75
           1       0.00      0.00      0.00        19

    accuracy                           0.80        94
   macro avg       0.40      0.50      0.44        94
weighted avg       0.64      0.80      0.71        94



In [10]:
X, y, data = load_thoracic()
X_train, y_train, X_test, y_test = split_data(X, y, pca_included = True, smote_included = False)
predictions = train_randomforest(X_train, y_train, X_test, y_test, use_randomsearch = False)

Model Assumptions:
	 The dataset used PCA for dimensionality reduction.
	 The dataset did NOT use SMOTE.
	 This model did NOT cross-validate through Random Search.
Confusion Matrix: 
[[75  0]
 [19  0]]
              precision    recall  f1-score   support

          -1       0.80      1.00      0.89        75
           1       0.00      0.00      0.00        19

    accuracy                           0.80        94
   macro avg       0.40      0.50      0.44        94
weighted avg       0.64      0.80      0.71        94



In [11]:
X, y, data = load_thoracic()
X_train, y_train, X_test, y_test = split_data(X, y, pca_included = False, smote_included = True)
predictions = train_randomforest(X_train, y_train, X_test, y_test, use_randomsearch = True)

Model Assumptions:
	 The dataset used SMOTE to rectify class imbalance.
	 This model has been cross-validated through Random Search.
{'n_estimators': 2000, 'min_samples_split': 3, 'max_depth': 17, 'criterion': 'gini'}
Confusion Matrix: 
[[69  6]
 [18  1]]
              precision    recall  f1-score   support

          -1       0.79      0.92      0.85        75
           1       0.14      0.05      0.08        19

    accuracy                           0.74        94
   macro avg       0.47      0.49      0.46        94
weighted avg       0.66      0.74      0.70        94



In [12]:
X, y, data = load_thoracic()
X_train, y_train, X_test, y_test = split_data(X, y, pca_included = True, smote_included = True)
predictions = train_randomforest(X_train, y_train, X_test, y_test, use_randomsearch = True)

Model Assumptions:
	 The dataset used PCA for dimensionality reduction.
	 The dataset used SMOTE to rectify class imbalance.
	 This model has been cross-validated through Random Search.
{'n_estimators': 2000, 'min_samples_split': 4, 'max_depth': 18, 'criterion': 'gini'}
Confusion Matrix: 
[[64 11]
 [16  3]]
              precision    recall  f1-score   support

          -1       0.80      0.85      0.83        75
           1       0.21      0.16      0.18        19

    accuracy                           0.71        94
   macro avg       0.51      0.51      0.50        94
weighted avg       0.68      0.71      0.70        94



## Cervical Cancer Dataset

In [35]:
data = pd.read_csv('../data/sobar-72.csv', sep = ',', header = 0)
X = data.iloc[:, 0:19].to_numpy()
y = data.iloc[:, 19].to_numpy()
X_train, y_train, X_test, y_test = split_data(X, y, pca_included = False, smote_included = False)
predictions = train_randomforest(X_train, y_train, X_test, y_test, use_randomsearch = False)

Model Assumptions:
	 The dataset did NOT use SMOTE.
	 This model did NOT cross-validate through Random Search.
Confusion Matrix: 
[[8 0]
 [2 5]]
              precision    recall  f1-score   support

           0       0.80      1.00      0.89         8
           1       1.00      0.71      0.83         7

    accuracy                           0.87        15
   macro avg       0.90      0.86      0.86        15
weighted avg       0.89      0.87      0.86        15



In [9]:
data = pd.read_csv('../data/sobar-72.csv', sep = ',', header = 0)
X = data.iloc[:, 0:19].to_numpy()
y = data.iloc[:, 19].to_numpy()
X_train, y_train, X_test, y_test = split_data(X, y, pca_included = True, smote_included = False)
predictions = train_randomforest(X_train, y_train, X_test, y_test, use_randomsearch = True)

Model Assumptions:
	 The dataset used PCA for dimensionality reduction.
	 The dataset did NOT use SMOTE.
	 This model has been cross-validated through Random Search.
{'n_estimators': 2000, 'min_samples_split': 2, 'max_depth': 12, 'criterion': 'gini'}
Confusion Matrix: 
[[8 0]
 [1 6]]
              precision    recall  f1-score   support

           0       0.89      1.00      0.94         8
           1       1.00      0.86      0.92         7

    accuracy                           0.93        15
   macro avg       0.94      0.93      0.93        15
weighted avg       0.94      0.93      0.93        15

